In [1]:
from osgeo import gdal



## defined the resolution by human

In [2]:
# Open the input image
input_file = r"F:\wenqu\trait_map\site2a_la_trait_map_project3.tif"
input_dataset = gdal.Open(input_file)

# Define the output file name and resolution
output_file = r"F:\wenqu\trait_map\site2a_la_trait_map_project3_resample.tif"
output_resolution = 5  # meters

# Get the input image size and resolution
cols = input_dataset.RasterXSize
rows = input_dataset.RasterYSize
input_geotransform = input_dataset.GetGeoTransform()
x_res = input_geotransform[1]
y_res = -input_geotransform[5]  # pixel size is negative, so flip it

# Compute the new output image size and resolution
x_scale = x_res / output_resolution
y_scale = y_res / output_resolution
new_cols = int(cols * x_scale)
new_rows = int(rows * y_scale)

# Create the output dataset
driver = gdal.GetDriverByName("GTiff")
output_dataset = driver.Create(output_file, new_cols, new_rows, input_dataset.RasterCount, input_dataset.GetRasterBand(1).DataType)

# Set the output geotransform
output_geotransform = (input_geotransform[0], output_resolution, 0, input_geotransform[3], 0, -output_resolution) 
output_dataset.SetGeoTransform(output_geotransform)

# Perform the resampling
gdal.ReprojectImage(input_dataset, output_dataset, input_dataset.GetProjection(), output_dataset.GetProjection(), gdal.GRA_Bilinear)

# Close the datasets
input_dataset = None
output_dataset = None


# copy another image resolution

In [8]:
src_dataset = gdal.Open(r'E:\wenqu\plant_biomass\plant_agb_5n.tif', gdal.GA_ReadOnly)
src_resolution_x = src_dataset.GetGeoTransform()[1]
src_resolution_y = src_dataset.GetGeoTransform()[5]

target_dataset = gdal.Open(r'E:\wenqu\segmentationn\masked_aviris_trait\site3b_pc_ymax.tif', gdal.GA_ReadOnly)
target_resolution_x = target_dataset.GetGeoTransform()[1]
target_resolution_y = target_dataset.GetGeoTransform()[5]

ratio_x = src_resolution_x / target_resolution_x
ratio_y = src_resolution_y / target_resolution_y
resample_method = gdal.GRA_Bilinear  # Choose the resampling method
output_width = target_dataset.RasterXSize
output_height = target_dataset.RasterYSize
output_dataset = gdal.GetDriverByName('GTiff').Create(r'E:\wenqu\plant_biomass\clipped_plant_agb\site3b_plant_agb.tif', output_width, output_height, src_dataset.RasterCount, src_dataset.GetRasterBand(1).DataType)
output_dataset.SetGeoTransform(target_dataset.GetGeoTransform())
output_dataset.SetProjection(target_dataset.GetProjection())
gdal.ReprojectImage(src_dataset, output_dataset, None, None, resample_method)
src_dataset = None
target_dataset = None
output_dataset = None
